# AI101 – Assignment 2: Finance Data Analysis
**Dataset:** `finance_data.csv`  
This notebook explores a financial dataset to understand its structure, clean it, and derive basic statistical insights.

---
## Section 1: Importing Libraries

Before we can analyze any data, we need to import the Python libraries that provide the tools we'll use throughout this notebook.

- **`pandas` (imported as `pd`)**: The primary library for data manipulation. It introduces the `DataFrame` — a table-like structure that makes reading, filtering, and transforming data easy.
- **`numpy` (imported as `np`)**: A numerical computing library. It provides fast math operations, array support, and special values like `NaN` (Not a Number), which is used to represent missing data.
- **`matplotlib.pyplot` (imported as `plt`)**: A plotting library that allows us to create charts and graphs to visualize data trends.
- **`seaborn` (imported as `sns`)**: Built on top of `matplotlib`, seaborn makes it easier to create attractive statistical visualizations with less code.

The `%matplotlib inline` magic command tells Jupyter to display any plots directly inside the notebook, rather than opening a separate window.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---
## Section 2: Loading the Dataset

Here we load the `finance_data.csv` file into a pandas `DataFrame` called `df`. A DataFrame is essentially a table with named columns and indexed rows — think of it as a spreadsheet inside Python.

- **`pd.read_csv('finance_data.csv')`**: Reads the CSV (Comma-Separated Values) file from the current working directory and parses it into a DataFrame. Each column in the CSV becomes a column in the DataFrame.
- **`df.head()`**: Displays the **first 5 rows** of the DataFrame. This is the standard first step when exploring a new dataset — it lets us quickly see the column names, data types, and what actual values look like. By default, `head()` shows 5 rows; you can pass a number (e.g., `df.head(10)`) to see more.

**What the output represents:** A table showing the first five records of the finance dataset. This gives us a preview of the columns available (e.g., dates, amounts, categories) and helps us understand what kind of data we're working with before we dive deeper.

In [ ]:
df = pd.read_csv('finance_data.csv')
df.head()

---
## Section 3: Exploring the Shape and Structure

Now we examine the overall size and structure of our dataset before doing any analysis.

- **`df.shape`**: Returns a tuple `(rows, columns)`. For example, `(50000, 8)` means the dataset has 50,000 records and 8 columns. This tells us the scale of the data we're working with.
- **`df.dtypes`**: Shows the **data type** of each column — for example, `int64` (whole numbers), `float64` (decimal numbers), or `object` (text/string). Knowing data types is critical because some operations only work on specific types (e.g., you cannot compute an average on a text column).
- **`df.info()`**: Provides a concise summary of the entire DataFrame: the number of rows, each column name, how many non-null (non-missing) values exist in each column, and the data type. This is extremely useful for spotting missing data and confirming everything loaded correctly.

**What the output represents:** A structural overview of the dataset. It tells us how large the data is, what each column contains, and whether any columns have missing values that need to be addressed during cleaning.

In [ ]:
print("Shape:", df.shape)
print("\nData Types:")
print(df.dtypes)
print("\nDataset Info:")
df.info()

---
## Section 4: Checking for Missing Values

Real-world datasets almost always have missing or null values. Before analyzing data, we must understand how much is missing and decide how to handle it.

- **`df.isnull()`**: Returns a DataFrame of the same shape as `df`, where each cell contains `True` if the value is missing (`NaN`) and `False` otherwise.
- **`.sum()`**: Counts the number of `True` values in each column, giving us the **total number of missing values per column**.
- **`df.isnull().sum() / len(df) * 100`**: Calculates the **percentage** of missing values in each column. Dividing the missing count by the total number of rows (`len(df)`) and multiplying by 100 converts it to a percentage. This helps us decide whether to drop a column (e.g., if 80% is missing) or fill it in.

**What the output represents:** A column-by-column count and percentage of missing data. For example, if the `income` column shows `1500 (3.0%)`, it means 1,500 rows are missing an income value, which is 3% of the total dataset. We use this to decide our data cleaning strategy.

In [ ]:
missing = df.isnull().sum()
missing_pct = df.isnull().sum() / len(df) * 100

missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])

---
## Section 5: Cleaning the Data

Data cleaning ensures our dataset is consistent and ready for analysis. Raw data often contains errors, duplicates, and missing values.

- **`df.drop_duplicates(inplace=True)`**: Removes any rows that are **exact duplicates** of another row. The `inplace=True` argument modifies the DataFrame directly instead of returning a new one. Duplicate records can skew analysis results — for example, a transaction counted twice would inflate totals.
- **`df.dropna(subset=[...])`** *(if used)*: Drops rows where **specific critical columns** are missing. We only drop rows where essential columns are null — columns that are less critical might be filled in instead.
- **`df.fillna(df.median())`** or **`df[col].fillna(df[col].mean())`**: Fills missing numerical values with the **median** or **mean** of that column. The median is often preferred over the mean because it is less sensitive to extreme outliers (e.g., one very large transaction wouldn't distort the median as much).
- **`df.reset_index(drop=True)`**: After removing rows, the index numbers may have gaps (e.g., 0, 1, 4, 5...). This resets them to a clean sequential index (0, 1, 2, 3...). `drop=True` prevents the old index from being added as a column.

**What the output represents:** Confirmation of how many rows/columns remain after cleaning, and that the data is now free of duplicates and handled missing values — ready for reliable analysis.

In [ ]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Fill missing numeric values with column median
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

print("Cleaned dataset shape:", df.shape)
print("Remaining missing values:", df.isnull().sum().sum())

---
## Section 6: Descriptive Statistics

Descriptive statistics give us a high-level numerical summary of the dataset, helping us understand the distribution and spread of values.

- **`df.describe()`**: Automatically calculates summary statistics for every **numerical column** in the DataFrame. The statistics it returns include:
  - **`count`**: How many non-missing values exist in the column.
  - **`mean`**: The average value across all rows.
  - **`std`** (standard deviation): How spread out the values are around the mean. A high std means values vary a lot; a low std means they cluster tightly around the mean.
  - **`min`** / **`max`**: The smallest and largest values in the column.
  - **`25%`, `50%`, `75%`** (quartiles): These divide the data into four equal parts. The `50%` value is the **median** — the middle value. The gap between `25%` and `75%` (the interquartile range) tells us where the middle 50% of data lives.

**What the output represents:** A comprehensive statistical snapshot of every numeric column. For a finance dataset, this might show the average transaction amount, the range of income values, and whether there are extreme outliers pulling the mean away from the median.

In [ ]:
df.describe()

---
## Section 7: Distribution of a Key Numeric Variable

Visualizing the distribution of a key variable helps us understand how values are spread — whether they're normally distributed, skewed, or contain outliers.

- **`plt.figure(figsize=(10, 5))`**: Creates a new figure (canvas) for the plot, with a width of 10 inches and height of 5 inches. Setting the figure size ensures the chart is large and readable.
- **`sns.histplot(df[col], kde=True, bins=40, color='steelblue')`**: Draws a **histogram** of a numeric column.
  - A histogram groups values into "bins" (ranges) and counts how many values fall in each bin, showing the frequency distribution.
  - **`kde=True`**: Overlays a **Kernel Density Estimate** curve — a smooth line that estimates the probability distribution of the data. It's like a smoothed version of the histogram.
  - **`bins=40`**: The data is split into 40 equal-width bins. More bins = more detail; fewer bins = broader overview.
- **`plt.title(...)`, `plt.xlabel(...)`, `plt.ylabel(...)`**: Add a title and axis labels to make the chart self-explanatory.
- **`plt.show()`**: Renders and displays the completed plot.

**What the output represents:** A histogram showing how frequently different value ranges appear in the data. For example, if plotting income, a right-skewed distribution would indicate most people have lower incomes with a few very high earners — which is typical of real-world finance data.

In [ ]:
# Plot distribution of the first numeric column
num_col = numeric_cols[0]

plt.figure(figsize=(10, 5))
sns.histplot(df[num_col], kde=True, bins=40, color='steelblue')
plt.title(f'Distribution of {num_col}')
plt.xlabel(num_col)
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

---
## Section 8: Correlation Heatmap

A correlation heatmap shows us how strongly pairs of numeric variables are related to each other. This is a foundational step before building any predictive model.

- **`df.corr()`**: Computes the **Pearson correlation coefficient** between every pair of numeric columns. The result is a matrix where each cell contains a value between **-1 and 1**:
  - **1.0**: Perfect positive correlation (as one variable increases, the other increases proportionally).
  - **-1.0**: Perfect negative correlation (as one increases, the other decreases).
  - **0**: No linear relationship.
  - Values closer to ±1 indicate stronger relationships.
- **`sns.heatmap(..., annot=True, cmap='coolwarm', fmt='.2f')`**: Visualizes the correlation matrix as a color-coded grid.
  - **`annot=True`**: Displays the actual correlation number inside each cell.
  - **`cmap='coolwarm'`**: Uses a red-blue color scale — red indicates positive correlation, blue indicates negative.
  - **`fmt='.2f'`**: Formats the numbers to 2 decimal places for readability.
  - **`linewidths=0.5`**: Adds thin lines between cells to improve readability.

**What the output represents:** A grid of color-coded correlation values. In a finance dataset, for example, a high positive correlation between `credit_score` and `loan_approval` would suggest that higher credit scores are associated with higher approval rates — a relationship worth investigating further.

In [ ]:
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f',
            linewidths=0.5, square=True)
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()

---
## Section 9: Analyzing Categorical Variables

Not all columns contain numbers. Categorical columns contain text labels (e.g., "Male/Female", "Yes/No", "City names"). Here we explore the distribution of those categories.

- **`df.select_dtypes(include='object').columns`**: Selects only columns that have the `object` data type, which in pandas typically means **text/string columns** (i.e., categorical data).
- **`df[col].value_counts()`**: For a given column, counts how many times each unique value appears, and returns them in **descending order** (most common first). This is the fundamental way to understand the composition of a categorical variable.
- **`sns.countplot(...)`**: Draws a bar chart where each bar represents a category and its height represents the count. This makes it visually easy to compare the frequency of each category.
- **`plt.xticks(rotation=45)`**: Rotates the x-axis labels by 45 degrees so that longer category names don't overlap and remain readable.

**What the output represents:** Bar charts for categorical columns showing how the data is distributed across categories. For example, in a finance dataset a `loan_purpose` column might show that most loans are for "debt consolidation" while very few are for "education" — this kind of insight informs further targeted analysis.

In [ ]:
cat_cols = df.select_dtypes(include='object').columns
print(f"Categorical columns: {list(cat_cols)}\n")

for col in cat_cols[:3]:  # Show up to 3 categorical columns
    print(f"--- {col} ---")
    print(df[col].value_counts(), "\n")

    plt.figure(figsize=(10, 4))
    order = df[col].value_counts().index[:15]  # Top 15 categories
    sns.countplot(data=df, x=col, order=order, palette='viridis')
    plt.title(f'Value Counts: {col}')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

---
## Section 10: Outlier Detection with Box Plots

Outliers are data points that are unusually far from the rest of the data. In finance, outliers could represent fraud, data entry errors, or genuinely unusual transactions. Box plots are a standard tool for spotting them.

- **Box plot anatomy**: A box plot shows five key statistics:
  - The **box** spans from the **25th percentile (Q1)** to the **75th percentile (Q3)** — this is the Interquartile Range (IQR), containing the middle 50% of data.
  - The **line inside the box** is the **median (50th percentile)**.
  - The **whiskers** extend to the furthest values within 1.5× the IQR from Q1 and Q3.
  - Any **dots beyond the whiskers** are classified as **outliers**.
- **`sns.boxplot(data=df, y=col)`**: Creates a vertical box plot for a numeric column.
- **Why this matters**: If the `amount` column has many extreme outliers, they could skew mean-based calculations and mislead predictive models. Outliers need to be investigated — removed, capped, or treated as a separate segment.

**What the output represents:** A visual summary of a column's distribution and any extreme values. Dots appearing far above the top whisker (or below the bottom) are outliers. In a finance dataset, a single extremely high transaction amount appearing as an outlier should be investigated — is it legitimate, or an error?

In [ ]:
fig, axes = plt.subplots(1, min(3, len(numeric_cols)), figsize=(14, 5))

if len(numeric_cols) == 1:
    axes = [axes]

for ax, col in zip(axes, numeric_cols[:3]):
    sns.boxplot(data=df, y=col, ax=ax, color='lightcoral')
    ax.set_title(f'Box Plot: {col}')

plt.suptitle('Outlier Detection via Box Plots', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Section 11: Summary and Key Takeaways

This final section summarizes what we learned from the finance dataset analysis.

- **`df.shape`**: We print the final cleaned shape of the DataFrame to confirm how many rows and columns remain after preprocessing.
- **`df.describe().T`**: We transpose (`.T`) the descriptive statistics table so that columns become rows — this makes it easier to read when there are many columns, as all stats for one variable are on one line.
- **Key insights recorded below**: Based on the visualizations and statistics above, we document the main findings — things like which variables have the widest spread, what the most common categories are, and which pairs of variables are most correlated.

**What the output represents:** A final clean summary of the dataset's key statistical properties after cleaning and exploration. This serves as the foundation for the next phase of the project — feature engineering and building a predictive model with the more complex dataset in Assignment 2a.

In [ ]:
print("=" * 50)
print("FINAL DATASET SUMMARY")
print("=" * 50)
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print(f"Numeric Features: {len(numeric_cols)}")
print(f"Categorical Features: {len(cat_cols)}")
print(f"Missing Values Remaining: {df.isnull().sum().sum()}")
print("\nDescriptive Statistics (transposed):")
df.describe().T